# Coordination number

Let's use a simple hdf5 file to compute the coordination number for a given time step
using a single file without loading LMGC90.

First thing is to load the *h5py* module and to read the file :

In [ ]:
import h5py
hf = h5py.File('../lmgc90.h5')

## Preparative work

Now, you can just trust the author and run the next section to generate a dictionary
allowing to map the name of some parameters to an integer id used within LMG90's core.

If you are interested in understanding how these data are obtained, you may need to
read the HDF5_basis notebook.

In [ ]:
basepath = hf['Help/parameters']
parameters = {}
for k in basepath.keys() :
    parameters[k] = dict( zip( map(bytes.decode,basepath[k+'/name'][()]), basepath[k+'/id'][()] ) )

In [ ]:
print( parameters.keys() )
parameters['bdyty']

In the same way, one can generate the dictionary in the reverse way with :

In [ ]:
rev_parameters = {}
for k in basepath.keys() :
    rev_parameters[k] = dict( zip( basepath[k+'/id'][()],  map(bytes.decode,basepath[k+'/name'][()]) ) )

In [ ]:
rev_parameters['bdyty']

Now following this, to extract data from *idata* and *rdata* array,
let's generate more dictionaries

In [ ]:
ik = {}
for k in hf['Help/VlocRloc/idata'].keys() :
    ik[k] = ( bytes.decode( hf['Help/VlocRloc/idata/'+k+'/name'][()] ),
              hf['Help/VlocRloc/idata/'+k+'/bound'][0]-1,
              hf['Help/VlocRloc/idata/'+k+'/bound'][1]-1
            )

rk = {}
for k in hf['Help/VlocRloc/rdata'].keys() :
    rk[k] = ( bytes.decode( hf['Help/VlocRloc/rdata/'+k+'/name'][()] ),
              hf['Help/VlocRloc/rdata/'+k+'/bound'][0]-1,
              hf['Help/VlocRloc/rdata/'+k+'/bound'][1]-1
            )

In [ ]:
ik

In [ ]:
rk

## Adjacence Map generation

First thing is to check that the version of the hdf5 file of
LMGC90 is compatible with this notebook code:

In [ ]:
assert(hf['version'][()] == 1), '[ERROR] wrong version of LMGC90 hdf5 file'

Now let's choose a record number to extract all *VlocRloc* data of
this corresponding time step :

In [ ]:
nb_record = hf['Simulation/nb_record'][()]
i_record  = 2
assert( i_record <= nb_record )

basepath = hf["Evolution/ID_"+str(i_record)]
i_step   = hf["Evolution/ID_"+str(i_record)+"/NStep"][()]
idata =  basepath['VlocRloc/idata'][()]
rdata =  basepath['VlocRloc/rdata'][()]

# really really paranoid
assert( idata.shape[0] == rdata.shape[0] )

print( "time step", i_step, " -> nb_inter = ", idata.shape[0] )

Since all the desired data have been extracted from the file. Do not forget to close !

In [ ]:
hf.close()

Finally it is needed to generate an adjacence map of the interactions :

In [ ]:
from collections import defaultdict
adj_map = defaultdict( list )

idx_cd_tactype = ik['tactype'][1]
idx_an_tactype = ik['tactype'][2]

# going through all interactions
for inter_i, inter_r in zip(idata,rdata) :

    # skipping antagonists of type JONCx
    if inter_i[ idx_an_tactype ] == parameters['tactype']['JONCx'] :
        #print( inter_i[1], ' has an antagonist JONCx... skipping' )
        continue

    # a contactor is identified by its type (DISKx, JONCx, etc) and its index in this type
    cd = ( inter_i[ idx_cd_tactype ], inter_i[ ik['itacty'][1] ] )
    an = ( inter_i[ idx_an_tactype ], inter_i[ ik['itacty'][2] ] )

    # if it is a new candidate/antgoniste pair
    if cd not in adj_map.keys() or an not in adj_map[cd] :
        # and if Rn > 0.
        idx_Rn = rk['rl'][2]
        if inter_r[ idx_Rn ] > 0. :
            adj_map[ cd ].append( an )
            adj_map[ an ].append( cd )

In this particular case, one can notice that, only interactions
with a positive normal reaction are kept.

Of course, any kind of criteria can be used instead of this.

Furthermore, this code exclude the *JONCx* antagonist, but it
is possible to change to count only one type of contactor or
to exclude more types at will.

In the end, *adj_map* is a dictionnary with a contactor as keys,
and a list of contators as values. To compute the number of coordinations,
it is needed to count for a single contactor, the number of associated
contactor (which is the length of the list in *adj_map* values).

## Coordination number account
This can be quickly done with :

In [ ]:
coordination_number = { contactor[1] : len(adjac) for contactor, adjac in adj_map.items() }

Now that, for a single contactor, the coordination number is stored,
some simple manipulation using *numpy* package allows to compute
the histogram of the number of particles with the same coordination number :

In [ ]:
import numpy as np
nbc = np.array( [*coordination_number.values()] )
nbc_hist = np.unique(nbc,return_counts=True)
print( nbc_hist )

Or directly for a visualization :

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.hist( nbc, bins=np.max(nbc) )

## Redo adjacence map

The adjacence map build earlier is build upon the contactors.
Since there are no cluster in this example, it should be alright.
But let's redo the adjacence map using body number instead of a
couple made of contactor type and index.

It is assumed that there are only rigid bodies in this case
and that any other type of entry will be skipped.

So the earlier code must be changed a little :

In [ ]:
from collections import defaultdict
adj_map = defaultdict( list )

idx_cd_tactype = ik['tactype'][1]
idx_an_tactype = ik['tactype'][2]
idx_cd_bdytype = ik['bdyty'][1]
idx_an_bdytype = ik['bdyty'][2]

# going through all interactions
for inter_i, inter_r in zip(idata,rdata) :

    # skipping antagonists of type JONCx
    # and any type of body whic is not a RBDY2
    if (   inter_i[ idx_an_tactype ] == parameters['tactype']['JONCx']
        or inter_i[ idx_cd_bdytype ] != parameters['bdyty']['RBDY2']
        or inter_i[ idx_an_bdytype ] != parameters['bdyty']['RBDY2']
       ) :
        continue

    cd = inter_i[ ik['ibdyty'][1] ]
    an = inter_i[ ik['itacty'][2] ]

    # if it is a new candidate/antgoniste pair
    if cd not in adj_map.keys() or an not in adj_map[cd] :
        # and if Rn > 0.
        idx_Rn = rk['rl'][2]
        if inter_r[ idx_Rn ] > 0. :
            adj_map[ cd ].append( an )
            adj_map[ an ].append( cd )

Finally the same bit of code to count the coordination number can be used:

In [ ]:
coordination_number = { body : len(adjac) for body, adjac in adj_map.items() }
nbc = np.array( [*coordination_number.values() ])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.hist( nbc, bins=np.max(nbc) )